In [2]:
import pandas as pd

In [3]:
import re
from googletrans import Translator

In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
df=pd.read_csv('topical_chat.csv')

In [6]:
df

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


In [7]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [8]:
df.head()

,conversation_id,message,sentiment
0,618,Yes we would. For something odd closer to ho...,Neutral
1,3103,"I have never seen that movie, though I heard ...",Neutral
2,1251,Hi there ! Looking forward to chat with You ! ;),Happy
3,7950,He also has a supermodel for a wife. That doe...,Happy
4,6928,I think they are all doing pretty well. In fa...,Happy


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188378 entries, 0 to 188377
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   conversation_id  188378 non-null  int64 
 1   message          188378 non-null  object
 2   sentiment        188378 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.3+ MB


In [10]:
df.sentiment.unique()

array([' Neutral', ' Happy', ' Curious to dive deeper', ' Surprised',
       ' Angry', ' Sad', ' Fearful', ' Disgusted'], dtype=object)

In [11]:
def basic_clean(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"@\w+", '', text)
    text = re.sub(r"#\w+", '', text)
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"\d+", '', text)
    text = re.sub(r"\s+", ' ', text).strip()
    return text

In [12]:
import string 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Sanket
[nltk_data]     Sinha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
port_stem=PorterStemmer()

In [52]:
stop_words=set(stopwords.words('english'))
def text_processing(mess):
    words=re.sub(r'[^a-zA-Z]',' ',mess).lower()
    cleaned_word=[port_stem.stem(word) for word in words.split() if word not in stop_words]
    return ' '.join(cleaned_word)

In [ ]:
stop_words

In [53]:
print(text_processing("Yes we would.  For something odd closer to home we've got Venus with it's no seasons."))

ye would someth odd closer home got venu season


In [56]:
df['cleaned_text']=df['message'].apply(text_processing)

In [57]:
df.head()

,conversation_id,message,sentiment,merge_sentiment,cleaned_text
0,618,Yes we would. For something odd closer to ho...,Neutral,Neutral,ye would someth odd closer home got venu season
1,3103,"I have never seen that movie, though I heard ...",Neutral,Neutral,never seen movi though heard quit funni comedi...
2,1251,Hi there ! Looking forward to chat with You ! ;),Happy,Happy,hi look forward chat
3,7950,He also has a supermodel for a wife. That doe...,Happy,Happy,also supermodel wife hurt especi sinc make
4,6928,I think they are all doing pretty well. In fa...,Happy,Happy,think pretti well fact like three show


In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [14]:
cv=CountVectorizer(analyzer=text_processing).fit(df['message'])
X=cv.transform(df['message'])

In [17]:
X.shape

(188378, 23759)

In [15]:
tfidf_transformer = TfidfTransformer().fit(X)
tfidf4 = tfidf_transformer.transform(X)

In [16]:
print(tfidf4.shape)

(188378, 23759)


In [17]:
from sklearn.naive_bayes import MultinomialNB

In [18]:
msg_train, msg_test, label_train, label_test = train_test_split(df['message'], df['sentiment'], stratify=df['sentiment'],test_size=0.2,random_state=42)

In [19]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_processing)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [22]:
pipeline.fit(msg_train,label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_processing at 0x0000024E3AE52660>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [23]:
predictions = pipeline.predict(msg_test)

In [24]:
predictions

array([' Curious to dive deeper', ' Curious to dive deeper',
       ' Curious to dive deeper', ..., ' Curious to dive deeper',
       ' Curious to dive deeper', ' Curious to dive deeper'], dtype='<U23')

In [25]:
print(classification_report(predictions,label_test))

C:\Users\Sanket Sinha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                         precision    recall  f1-score   support

                  Angry       0.00      0.00      0.00         0
 Curious to dive deeper       0.96      0.45      0.61     34804
              Disgusted       0.00      0.00      0.00         0
                Fearful       0.00      0.00      0.00         0
                  Happy       0.07      0.51      0.12       776
                Neutral       0.10      0.47      0.16      1708
                    Sad       0.00      0.00      0.00         0
              Surprised       0.03      0.45      0.05       388

               accuracy                           0.45     37676
              macro avg       0.14      0.23      0.12     37676
           weighted avg       0.89      0.45      0.57     37676



C:\Users\Sanket Sinha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Sanket Sinha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
print(df['sentiment'].value_counts())

sentiment
Curious to dive deeper    80888
Neutral                   41367
Surprised                 30638
Happy                     29617
Sad                        2533
Disgusted                  1433
Fearful                    1026
Angry                       876
Name: count, dtype: int64


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [19]:
tfcv=TfidfVectorizer(ngram_range=(1,2), max_features=50000)

In [20]:
msg_train, msg_test, label_train, label_test =train_test_split(df['message'], df['sentiment'], test_size=0.40,stratify=df['sentiment'])

In [21]:
new_data=tfcv.fit_transform(msg_train)

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
model = LogisticRegression(max_iter=1000, class_weight='balanced')

In [24]:
model.fit(new_data,label_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [25]:
pred_logistic=model.predict(tfcv.transform(msg_test))

In [26]:
print(classification_report(pred_logistic,label_test))

                         precision    recall  f1-score   support

                  Angry       0.16      0.06      0.08       947
 Curious to dive deeper       0.46      0.71      0.56     21097
              Disgusted       0.24      0.07      0.11      1974
                Fearful       0.28      0.09      0.14      1254
                  Happy       0.41      0.35      0.38     13860
                Neutral       0.42      0.44      0.43     15985
                    Sad       0.40      0.13      0.19      3122
              Surprised       0.51      0.37      0.43     17113

               accuracy                           0.45     75352
              macro avg       0.36      0.28      0.29     75352
           weighted avg       0.44      0.45      0.43     75352



In [27]:
df.head()

,conversation_id,message,sentiment
0,618,Yes we would. For something odd closer to ho...,Neutral
1,3103,"I have never seen that movie, though I heard ...",Neutral
2,1251,Hi there ! Looking forward to chat with You ! ;),Happy
3,7950,He also has a supermodel for a wife. That doe...,Happy
4,6928,I think they are all doing pretty well. In fa...,Happy


In [28]:
def label_merge(text):
    if text.strip() in ['Sad','Disgusted','Fearful','Angry']:
        return 'Negative'
    else:
        return text 

In [29]:
df['merge_sentiment']=df['sentiment'].apply(label_merge)

In [30]:
df

,conversation_id,message,sentiment,merge_sentiment
0,618,Yes we would. For something odd closer to ho...,Neutral,Neutral
1,3103,"I have never seen that movie, though I heard ...",Neutral,Neutral
2,1251,Hi there ! Looking forward to chat with You ! ;),Happy,Happy
3,7950,He also has a supermodel for a wife. That doe...,Happy,Happy
4,6928,I think they are all doing pretty well. In fa...,Happy,Happy
...,...,...,...,...
188373,5490,I was aware of most of that. I knew it appear...,Curious to dive deeper,Curious to dive deeper
188374,4748,"I do, I love music videos, what about you?",Happy,Happy
188375,6042,It definitely seems that he'd be the angry ty...,Curious to dive deeper,Curious to dive deeper
188376,6725,Yes and now those companies turned out to be ...,Surprised,Surprised


In [31]:
print(df['merge_sentiment'].value_counts())

merge_sentiment
 Curious to dive deeper    80888
 Neutral                   41367
 Surprised                 30638
 Happy                     29617
Negative                    5868
Name: count, dtype: int64


In [32]:
merge_msg_train, merge_msg_test, merge_label_train, merge_label_test =train_test_split(df['message'], df['merge_sentiment'], test_size=0.20,stratify=df['merge_sentiment'])

In [33]:
merged_new_data=tfcv.fit_transform(merge_msg_train)

In [34]:
model.fit(merged_new_data,merge_label_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [35]:
pred_logistic=model.predict(tfcv.transform(merge_msg_test))

In [36]:
print(classification_report(merge_label_test,pred_logistic))

                         precision    recall  f1-score   support

 Curious to dive deeper       0.71      0.48      0.57     16178
                  Happy       0.35      0.42      0.38      5923
                Neutral       0.43      0.43      0.43      8273
              Surprised       0.37      0.51      0.43      6128
               Negative       0.17      0.47      0.25      1174

               accuracy                           0.46     37676
              macro avg       0.41      0.46      0.41     37676
           weighted avg       0.52      0.46      0.48     37676



In [37]:
print(accuracy_score(merge_label_test,pred_logistic))

0.4622040556322327


In [38]:
from sklearn.svm import LinearSVC

In [39]:
svc_model = LinearSVC()

In [40]:
svc_model.fit(merged_new_data,merge_label_train)

LinearSVC()

In [41]:
pred_svc = svc_model.predict(tfcv.transform(merge_msg_test))

In [42]:
print(classification_report(merge_label_test,pred_svc))

                         precision    recall  f1-score   support

 Curious to dive deeper       0.57      0.70      0.63     16178
                  Happy       0.39      0.28      0.33      5923
                Neutral       0.43      0.41      0.42      8273
              Surprised       0.39      0.34      0.37      6128
               Negative       0.32      0.15      0.20      1174

               accuracy                           0.49     37676
              macro avg       0.42      0.38      0.39     37676
           weighted avg       0.48      0.49      0.48     37676



In [43]:
print(accuracy_score(merge_label_test,pred_svc))

0.49493045970909866


In [105]:
print(merge_msg_test.iloc[450],merge_label_test.iloc[450])

 Crazy to think about the 1.65 billion dollar price. But it has paid off for them with the ad revenue  Curious to dive deeper


In [106]:
print(svc_model.predict(tfcv.transform([merge_msg_test.iloc[450]])))

[' Happy']


In [45]:
from sklearn.pipeline import Pipeline

In [75]:
from sklearn.model_selection import GridSearchCV 

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('clf', LinearSVC())
])

# Set the grid of parameters to search
param_grid = {
    'tfidf__max_features': [10000],
    'tfidf__ngram_range': [(1, 2)],
    'clf__C': [0.1, 1, 10],
    'clf__class_weight': ['balanced']
}

from sklearn.model_selection import RandomizedSearchCV

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=4,                # Only test 4 random combinations
    cv=3,
    scoring='f1_weighted',
    verbose=2,
    n_jobs=-1,
    random_state=42
)

In [76]:
search.fit(merge_msg_train, merge_label_train)

C:\Users\Sanket Sinha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 3 is smaller than n_iter=4. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 3 candidates, totalling 9 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('clf', LinearSVC())]),
                   n_iter=4, n_jobs=-1,
                   param_distributions={'clf__C': [0.1, 1, 10],
                                        'clf__class_weight': ['balanced'],
                                        'tfidf__max_features': [10000],
                                        'tfidf__ngram_range': [(1, 2)]},
                   random_state=42, scoring='f1_weighted', verbose=2)

In [77]:
print("Best Parameters Found:")
print(search.best_params_)

Best Parameters Found:
{'tfidf__ngram_range': (1, 2), 'tfidf__max_features': 10000, 'clf__class_weight': 'balanced', 'clf__C': 0.1}


In [78]:
best_model = search.best_estimator_
y_pred = best_model.predict(merge_msg_test)

In [80]:
print(classification_report(merge_label_test, y_pred))

                         precision    recall  f1-score   support

 Curious to dive deeper       0.43      0.48      0.46     16178
                  Happy       0.16      0.12      0.13      5923
                Neutral       0.23      0.16      0.18      8273
              Surprised       0.17      0.12      0.14      6128
               Negative       0.03      0.13      0.05      1174

               accuracy                           0.29     37676
              macro avg       0.20      0.20      0.19     37676
           weighted avg       0.29      0.29      0.28     37676



In [98]:
import joblib

In [103]:
joblib.dump(svc_model, 'svc_model.pkl')

['svc_model.pkl']

In [104]:
joblib.dump(tfcv, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']